# Utlyste stillinger til Nav med sklearn

Det som gjøres er inspirert av tutorial [https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f](https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f)

Datasett er hentet fra [https://data.norge.no/organisasjoner/arbeids-og-velferdsetaten-nav](https://data.norge.no/organisasjoner/arbeids-og-velferdsetaten-nav)


Spørsmål tas gjerne imot hos vikfand@gmail.com

In [1]:
import os
import sys
import sklearn
import pandas as pd
import numpy as np
import xml
import re
from time import time
import matplotlib.pyplot as plt
from pprint import pprint
import html


print(sys.version)
data_dir = os.path.join('.', 'data')

3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]


In [2]:
from lib.data import download_data

# Download the data (will not download if it's already downloaded)
download_data(data_dir)

In [3]:
from lib.load_dataset import load_datasets

# Load datasets into a pandas DataFrame from .csv files
df = load_datasets(data_dir, 2015, 2015)

print(df.shape)
df.head()

Loading datasets from year 2015 to 2015
(118096, 24)


,stillingsnummer,nav_enhet_kode,registrert_dato,sistepubl_dato,statistikk_aar_mnd,offisiell_statistikk_flagg,stilling_kilde,arbeidssted_fylkesnummer,arbeidssted_fylke,arbeidssted_kommunenummer,...,yrke_grovgruppe,yrkeskode,yrke,yrkesbetegnelse,virksomhet_organisasjonsnr,virksomhet_navn,antall_stillinger,stillingstittel,statistikk_periode,stillingsbeskrivelse
0,101201412000010,101,18.12.2014,02.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Undervisning,2342,Førskolelærere,Førskolelærer,973967592,HØGSKOLEN I ØSTFOLD,1,Barnehagelærar / Førstekonsulent (14/04241),201501,# senter for framandspråk i opplæringa er det ...
1,101201412000011,101,19.12.2014,01.02.2015,201501,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Serviceyrker og annet arbeid,5413,Fengselsbetjenter,Fengselsbetjent,993315060,HALDEN FENGSEL,1,Halden fengsel - ledig fast stilling som verks...,201501,Jobbnorge ID: 108617<br><br><p dir=#LTR# align...
2,101201412000012,101,19.12.2014,16.01.2015,201501,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Serviceyrker og annet arbeid,5413,Fengselsbetjenter,Fengselsbetjent,993315060,HALDEN FENGSEL,1,Halden fengsel - ledig fast stilling som verks...,201501,Jobbnorge ID: 108301<br><br><p dir=#LTR# align...
3,101201412000013,101,23.12.2014,23.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Butikk- og salgsarbeid,3322,Selgere (engros),Key Account Manager (salg),886332882,SMART INNOVATION NORWAY AS,1,Key Account Manager,201501,# har ambisjoner om å vokse og styrker teamet ...
4,101201412000014,101,23.12.2014,30.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Ingeniør- og ikt-fag,2152,Sivilingeniører (elektronikk),Forsker (elektronikk),886332882,SMART INNOVATION NORWAY AS,1,Senior Reasearchers/Reasearchers,201501,# is now expanding its leading team with senio...


In [4]:
from lib.preprocessing import remove_tags, unescape_html, remove_whitespace

# Select subset of columns
col = [
    'stilling_kilde', 
    'stillingsnummer', 
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'arbeidssted_fylkesnummer',
    'arbeidssted_kommunenummer',
    'virksomhet_navn',
    'arbeidssted_fylke',
]
df = df[col]


# Preprocess text and add some columns
df['stillingsbeskrivelse'] = df['stillingsbeskrivelse'].map(remove_tags).map(unescape_html).map(remove_whitespace)
df['is_from_nav'] = df['stilling_kilde'].map(lambda x: x=='Reg av arb.giver på nav.no')
df['from_media'] = df['stilling_kilde'].map(lambda x: x=='Annonsert i media')
df['is_healthcare'] = df['yrke_grovgruppe'].map(lambda x: x=='Helse, pleie og omsorg')
df['is_industrial'] = df['yrke_grovgruppe'].map(lambda x: x=='Industriarbeid')
df['contains_nav'] = df['stillingsbeskrivelse'].map(lambda x: 'nav' in x)

print(df.shape)
df.head()




(118096, 13)


,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,is_healthcare,is_industrial,contains_nav
0,Annonsert i media,101201412000010,# senter for framandspråk i opplæringa er det ...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD,Østfold,False,True,False,False,False
1,Overført fra arbeidsgiver,101201412000011,Jobbnorge ID: 108617# fengsel er det ledig fas...,Serviceyrker og annet arbeid,01,0101,HALDEN FENGSEL,Østfold,False,False,False,False,False
2,Overført fra arbeidsgiver,101201412000012,Jobbnorge ID: 108301# fengsel er det ledig fas...,Serviceyrker og annet arbeid,01,0101,HALDEN FENGSEL,Østfold,False,False,False,False,False
3,Annonsert i media,101201412000013,# har ambisjoner om å vokse og styrker teamet ...,Butikk- og salgsarbeid,01,0101,SMART INNOVATION NORWAY AS,Østfold,False,True,False,False,False
4,Annonsert i media,101201412000014,# is now expanding its leading team with senio...,Ingeniør- og ikt-fag,01,0101,SMART INNOVATION NORWAY AS,Østfold,False,True,False,False,False


In [12]:
from sklearn.model_selection import train_test_split

data_subset_size = len(df) # Just use a small dataset when experimenting or it will take too much time
target_column_name = 'yrke_grovgruppe' # Which label to predict
random_seed = 305

# Make training and test sets of the data. 
def get_train_and_test_sets(df, sample_size=None, test_size=0.2, random_seed=305):
    if not sample_size:
        sample_size = len(df) // 10
    data_sample = df.sample(n=sample_size, random_state=random_seed)
    return train_test_split(
        data_sample, 
        data_sample[target_column_name],
        test_size=.2,
        random_state=random_seed
    )

X_train, X_test, y_train, y_test = get_train_and_test_sets(df)
X_train

,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn,arbeidssted_fylke,is_from_nav,from_media,is_healthcare,is_industrial,contains_nav
28849,Annonsert i media,321201503000035,# ligger sentralt til på Skullerud og har ca. ...,Undervisning,03,0301,OSLO VOKSENOPPLÆRING SKULLERUD,Oslo,False,True,False,False,False
86773,Fra Eures,1502201412000056,"Arbeidssted: Kommune i Møre og Romsdal fylke, ...","Helse, pleie og omsorg",12,1201,HELSE PERSONAL AS,Hordaland,False,False,True,False,False
79955,Annonsert i media,1219201502000039,Bømlo brannvern har ledig 100 % vikariat fram ...,Akademiske yrker,12,1219,BØMLO KOMMUNE BRANN OG BEREDSKAP,Hordaland,False,True,False,False,False
100626,Annonsert i media,1703201503000018,Akuttmottak er organisatorisk tilknyttet kirur...,"Helse, pleie og omsorg",17,1703,HELSE NORD-TRØNDELAG HF,Nord-Trøndelag,False,True,True,False,False
85658,Reg av arb.giver på nav.no,1432201506000024,For kunde søkjer vi erfarne og sjølvgående tøm...,Bygg og anlegg,14,1432,VIKAR AS,Sogn og Fjordane,True,False,False,False,False
114752,Annonsert i media,1939201501000006,Vi søker en tydelig og handlingsdyktig rådmann...,Ledere,19,1939,STORFJORD KOMMUNE DRIFTSJEFENS,Troms,False,True,False,False,False
49825,Annonsert i media,625201505000013,"Killingrud ungdomsskole har ledig vikariater, ...",Undervisning,06,0625,KILLINGRUD UNGDOMSSKOLE,Buskerud,False,True,False,False,False
86183,Annonsert i media,1439201510000006,Vi ønsker å ansette en person som i første omg...,Serviceyrker og annet arbeid,14,1443,ISOVENT HOLDING AS,Sogn og Fjordane,False,True,False,False,False
109316,Annonsert i media,1870201509000022,Klikk her for utlysningstekst.,Barne- og ungdomsarbeid,18,1870,VESTERÅLEN BARNEVERN,Nordland,False,True,False,False,False
100685,Annonsert i media,1703201505000018,Sykepleier 51% fast stilling Namsos bo- og vel...,"Helse, pleie og omsorg",17,1703,NAMSOS BO- OG SERVICESENTER,Nord-Trøndelag,False,True,True,False,False


In [17]:
# Make the pipeline that transforms our data and trains the classifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=.25)),
    ('svc', LinearSVC()),
])

pipeline.fit(X_train['stillingsbeskrivelse'].values, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.25, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [14]:
# Test the data by running the test data set on the trained pipeline
predicted = pipeline.predict(X_test['stillingsbeskrivelse'].values)
pprint(predicted)
np.mean(predicted == y_test)

array(['Helse, pleie og omsorg', 'Meglere og konsulenter',
       'Akademiske yrker', ..., 'Ledere', 'Undervisning', 'Undervisning'],
      dtype=object)


0.7480948348856901

In [15]:
from sklearn import metrics

# Print some stats of our results
print(metrics.classification_report(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))

                                    precision    recall  f1-score   support

                  Akademiske yrker       0.68      0.65      0.66       195
           Barne- og ungdomsarbeid       0.91      0.46      0.61        84
            Butikk- og salgsarbeid       0.82      0.86      0.84       171
                    Bygg og anlegg       0.68      0.68      0.68       114
            Helse, pleie og omsorg       0.81      0.95      0.88       670
                    Industriarbeid       0.51      0.44      0.47        81
Ingen yrkesbakgrunn eller uoppgitt       0.00      0.00      0.00        16
              Ingeniør- og ikt-fag       0.58      0.76      0.66       164
       Jordbruk, skogbruk og fiske       0.00      0.00      0.00         8
                      Kontorarbeid       0.77      0.56      0.65        97
                            Ledere       0.43      0.30      0.35        61
            Meglere og konsulenter       0.67      0.47      0.55        89
           

/opt/conda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
# Try the pipeline with some custom data

prediction = pipeline.predict(['difi leikanger'])
pprint(prediction)

array(['Ingeniør- og ikt-fag'], dtype=object)


In [10]:
# Analyze which words are most characteristic of each category

tfidf = TfidfVectorizer()

df_sample = df.sample(n=20000) # Will take very much time and memory if n>100000

features = tfidf.fit_transform(df_sample['stillingsbeskrivelse'])
labels = df_sample['yrke_grovgruppe']
features.shape

from sklearn.feature_selection import chi2
N = 10
for label in labels.drop_duplicates():
    features_chi2 = chi2(features, labels == label)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    print("# '{}':".format(label))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))


# 'Butikk- og salgsarbeid':
  . Most correlated unigrams:
. kundebehandling
. selgere
. butikkens
. du
. butikkmedarbeider
. butikken
. butikksjef
. selger
. butikk
. salg
# 'Helse, pleie og omsorg':
  . Most correlated unigrams:
. sykehjem
. lege
. helg
. vernepleier
. helsefagarbeider
. sykepleiere
. turnus
. autorisasjon
. pasienter
. sykepleier
# 'Ingeniør- og ikt-fag':
  . Most correlated unigrams:
. the
. of
. ikt
. sivilingeniør
. prosjektering
. teknisk
. it
. bioingeniør
. and
. ingeniør
# 'Reiseliv og transport':
  . Most correlated unigrams:
. restauranten
. kjøkkenet
. servitører
. kjøkken
. sjåfør
. carte
. restaurant
. servitør
. mat
. kokk
# 'Undervisning':
  . Most correlated unigrams:
. lærer
. undervisning
. barnehagen
. barnehagelærer
. elever
. rektor
. barnehage
. skole
. skolen
. pedagogisk
# 'Industriarbeid':
  . Most correlated unigrams:
. verksted
. sveising
. industrimekaniker
. energimontør
. mekanisk
. sveiser
. kjøretøy
. cnc
. mekaniker
. bilmekaniker
# 'K